# [Ned Batchelder: Getting Started Testing - PyCon 2014](https://youtu.be/FxSsnHeWQBY)

In [1]:
%%writefile portfolio1.py
class Portfolio(object):
    """A simple portfolio"""
    def __init__(self):
        # stocks is a list of lists:
        #   [[name, shares, price], ...]
        self.stocks = []
        
    def buy(self, name, shares, price):
        """Buy `name`: `shares` shares at `price`."""
        self.stocks.append([name,shares,price])
    
    def cost(self):
        """What was total cost of this portfolio?"""
        amt = 0.0
        for name, shares, price in self.stocks:
            amt += shares*price 
        return amt
    
    def sell(self, name, shares):
        """Sell some shares"""
        for holding in self.stocks:
            if holding[0] == name:
                if holding[1] < shares:
                    raise ValueError("Not enough shares")
                holding[1] -= shares
                break
        else:
            raise ValueError("You don't own that stock")
            

Overwriting portfolio1.py


In [2]:
from portfolio1 import Portfolio
myStocks = Portfolio()
print(myStocks.cost())
myStocks.buy('Telsa',2,424.87)
print(myStocks.cost())

0.0
849.74


## A simple unit test

In [3]:
%%writefile test_port1.py
import unittest
from portfolio1 import Portfolio

class PortfolioTest(unittest.TestCase):
    def test_buy_one_stock(self):
        p = Portfolio()
        p.buy("IBM", 100, 176.48)
        assert p.cost() == 17648.0

Overwriting test_port1.py


In [4]:
%%bash

python -m unittest test_port1

.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


Single dot in above means ran only one test. Everything went okay!

## Add more tests

In [7]:
%%writefile test_port2.py

import unittest 
from portfolio1 import Portfolio

class PortfolioTest(unittest.TestCase):
    def test_empty(self):
        p = Portfolio()
        assert p.cost() == 0
    
    def test_buy_one_stock(self):
        p = Portfolio()
        p.buy("IBM", 100, 176.48)
        assert p.cost() == 17648.0
        
    def test_buy_two_stocks(self):
        p = Portfolio()
        p.buy("IBM", 100, 176.48)
        p.buy("HPQ", 100, 36.15)
        self.assertEqual(p.cost(), 17648.0 + 3615.0)

Overwriting test_port2.py


In [8]:
%%bash

python -m unittest test_port2

...
----------------------------------------------------------------------
Ran 3 tests in 0.000s

OK


## Test sell()

In [13]:
%%writefile test_port3.py
import unittest 
from portfolio1 import Portfolio

class PortfolioSellTest(unittest.TestCase):
    def test_sell(self):
        p = Portfolio()
        p.buy("MSFT", 100, 27.0)
        p.buy("DELL", 100, 17.0)
        p.buy("ORCL", 100, 34.0)
        p.sell("MSFT", 50)
        self.assertEqual(p.cost(),6450)
    
    def test_not_enough(self):
        p = Portfolio() # Didn't I just do this?
        p.buy("MSFT", 100, 27.0)
        p.buy("DELL", 100, 17.0)
        p.buy("ORCL", 100, 34.0)
        with self.assertRaises(ValueError):
            p.sell("MSFT",200)
            
    def test_dont_own_it(self):
        p = Portfolio() # What, again??
        p.buy("MSFT", 100, 27.0)
        p.buy("DELL", 100, 17.0)
        p.buy("ORCL", 100, 34.0)
        with self.assertRaises(ValueError):
            p.sell("IBM",1)

Overwriting test_port3.py


In [14]:
%%bash

python -m unittest test_port3

...
----------------------------------------------------------------------
Ran 3 tests in 0.000s

OK


There is a lot of repetition of same code in class PortfolioSellTest.
Let's refactor it.

In [15]:
%%writefile test_port3.py
import unittest 
from portfolio1 import Portfolio

class PortfolioSellTest(unittest.TestCase):
    # The following setUp method is invoked before each test method
    def setUp(self):
        self.p = Portfolio()
        self.p.buy("MSFT", 100, 27.0)
        self.p.buy("DELL", 100, 17.0)
        self.p.buy("ORCL", 100, 34.0)
        
    def test_sell(self):
        self.p.sell("MSFT", 50)
        self.assertEqual(self.p.cost(),6450)
    
    def test_not_enough(self):
        with self.assertRaises(ValueError):
            self.p.sell("MSFT",200)
            
    def test_dont_own_it(self):
        with self.assertRaises(ValueError):
            self.p.sell("IBM",1)

Overwriting test_port3.py


In [16]:
%%bash

python -m unittest test_port3

...
----------------------------------------------------------------------
Ran 3 tests in 0.000s

OK
